In [1]:
# !pip install gym

In [2]:
# !pip install stable-baselines3[extra]

In [ ]:
# pip install --upgrade shimmy gymnasium

Note: you may need to restart the kernel to use updated packages.


In [4]:
import gymnasium as gym
import gym
import numpy as np
import random
from gym import spaces
import pandas as pd

In [5]:
df = pd.read_csv("Mental_Health_Score_Dataframe.csv")

In [6]:
df.head()

,Emotional_Score,Social_Media_Time_Score,Platform_Score,Usage_Pattern_Score,Impact_Score,Additional_Feature_Score,TFIDF_Score,Mental_Health_Score
0,84,13,20,18,27,38,0,200
1,56,13,10,2,0,35,0,116
2,97,23,19,18,61,34,0,252
3,114,13,19,11,54,63,0,274
4,87,13,19,18,25,10,0,172


In [7]:
class MentalHealthEnv(gym.Env):
    def __init__(self, df):
        super(MentalHealthEnv, self).__init__()

        #loading the dataset
        self.df = df.copy()
        self.current_index = 0


        #define state: 7 mental health scores (ranging between 0 to 300)
        self.observation_space = spaces.Box(low = 0, high = 300, shape = (7,), dtype = np.float32)

        # defining action space
        self.actions = [
            "Engage in Outdoor activities",
            "Reduce the social media usage",
            "Watch educational videos",
            "Increase mindfulness practicess like yoga",
            "Interact with positive communities",
            "Avoid stress-inducing content",
            "Follow a structured daily routine"
        ]

        self.action_space = spaces.Discrete(len(self.actions))

        #initiating state: random values between 50 and 250 (to avoid zero-starting issues)
        self.state = np.random.uniform(low = 50, high = 250, size = (7,))

        # define action effects (each action affects specific scores)
        self.action_effects = {
            "Engage in Outdoor activities": np.array([10, 5, 0, 8, -5, 7, 0]),
            "Reduce the social media usage": np.array([5, -15, 0, 0, 10, 0, 0]),
            "Watch educational videos": np.array([5, 0, 15, 7, -3, 10, 0]),
            "Increase mindfulness practicess like yoga": np.array([12, 0, 0, 5, -8, 15, 0]),
            "Interact with positive communities": np.array([6, 0, -8, 0, 12, 0, 0]),
            "Avoid stress-inducing content": np.array([6, 0, -8, 0, 12, 0, 0]),
            "Follow a structured daily routine": np.array([10, 0, 0, 0, 7, 8, 0])
        }
        self.state = self.get_state_from_df()


    def get_state_from_df(self):
        # extract the initial state from the dataframe based on the current index
        row = self.df.iloc[self.current_index]
        return np.array(row[:-1])
    
    def step(self, action):
        #apply an action and update mental health scores
        self.state = np.clip(self.state + self.action_effects[self.actions[action]], 0, 300)


        # reward: Improvement in mental health score 
        self.df.at[self.current_index, "Mental_Health_Score"] = np.sum(self.state)
        reward = self.df.at[self.current_index, "Mental_Health_Score"]

        #print(f"State Shape: {self.state.shape}")

        done = np.all(self.state >= 280) #stops if all the score are near 300
        
        return self.state, reward, done, {}

    def reset(self):
        """Move to the next user in the dataset or restart if finished."""
        self.current_index += 1  # Move to the next row
        if self.current_index >= len(self.df):
            self.current_index = 0  # Reset if all users are processed
        self.state = self.get_state_from_df()
        return self.state


In [8]:
env = MentalHealthEnv(df)

### Model Training

In [9]:
from stable_baselines3 import DQN, A2C

In [10]:
dqn_model = DQN("MlpPolicy", env, verbose=1)
dqn_model.learn(total_timesteps=5000)
dqn_model.save("mental_health_dqn_model")

# Train A2C
a2c_model = A2C("MlpPolicy", env, verbose=1)
a2c_model.learn(total_timesteps=5000)
a2c_model.save("mental_health_a2c_model")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\tejas\anaconda3\anaconda\envs\pytorch_env\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\tejas\anaconda3\anaconda\envs\pytorch_env\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


-------------------------------------
| time/                 |           |
|    fps                | 309       |
|    iterations         | 100       |
|    time_elapsed       | 1         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -1.9      |
|    explained_variance | -1.19e-07 |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | 6.44e+03  |
|    value_loss         | 1.56e+07  |
-------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 309      |
|    iterations         | 200      |
|    time_elapsed       | 3        |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -1.9     |
|    explained_variance | 1.19e-07 |
|    learning_rate      | 0.0007   |
|    n_updates          | 199      |
|    policy_loss        | 6.51e+03 |
|    value_loss         

In [ ]:
def evaluate_model(model, env, num_episodes=10, max_steps=100):
    total_rewards = []

    for _ in range(num_episodes):
        state = env.reset()
        done = False
        episode_reward = 0
        step_count = 0  # Track steps per episode

        while not done and step_count < max_steps: 
            action, _ = model.predict(state)
            state, reward, done, _ = env.step(action)
            episode_reward += reward
            step_count += 1

        total_rewards.append(episode_reward)

    return np.mean(total_rewards), np.std(total_rewards)


# Load trained models
dqn_model = DQN.load("mental_health_dqn_model")
a2c_model = A2C.load("mental_health_a2c_model")

# Evaluate both models
dqn_mean, dqn_std = evaluate_model(dqn_model, env)
a2c_mean, a2c_std = evaluate_model(a2c_model, env)

In [12]:
dqn_mean

np.float64(84305.2)

In [13]:
dqn_std

np.float64(2201.599227834167)

In [14]:
a2c_mean

np.float64(101014.2)

In [15]:
a2c_std

np.float64(7506.765732324408)